### Using Rbdyn and SpaceVecAlg
1. The library comes with python bindings using cython transcription. Thus, fast prototyping can be done to verify algorithms using the provide python functions.

2. To setup the libraries, we will need to install some common components first.
<br>`sudo apt install apt-transport-https lsb-release`

3. *Eigen3ToPython*
<br>1) `sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key 892EA6EE273707C6495A6FB6220D644C64666806`
<br>2) `sudo sh -c 'echo "deb https://dl.bintray.com/gergondet/multi-contact-release $(lsb_release -sc) main" | sudo tee -a /etc/apt/sources.list.d/multi-contact.list'`
<br>3) `sudo apt update && sudo apt install python-eigen python3-eigen`

4. *SpaceVecAlg*
<br>1) `sudo apt update && sudo apt install libspacevecalg-dev python-spacevecalg python3-spacevecalg`

5. *RBdyn*
<br>1) `sudo apt update && sudo apt install librbdyn-dev python-rbdyn python3-rbdyn`

#### Theory
The algorithm is based on Featherstone's spatial notation. In general, any articulated rigid body system can be captured by the following equation:
$$M \ddot q + h = S^T \tau + J_c^T F_c$$
where $S$ is the selection matrix for the active joints. For a system that is fully actuated, $S=I$. $J_c^T F_c$ is the torque contribution from the contact forces. $M$ and $h$ are respectively the mass matrix and the nonlinear effects (coriolis/centrifugal + gravity) terms.

In [3]:
# Simple cart pole example

import numpy as np
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation()
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row])
        
    return m4d

def print_multi_body(m):
    '''Helper function to display the contents of a multibody structure
    '''
    print("number of bodies:", m.nrBodies())
    print("number of joints:", m.nrJoints())
    
    print("bodies:")
    for bi, b in enumerate(m.bodies()):
        print("body index:", bi, "body name:", b.name().decode("utf-8"))
    
    print("joints:")
    for ji, j in enumerate(m.joints()):
        print("joint index:", ji, "joint name", j.name().decode("utf-8"))
    
# import robot as global variable
p = from_urdf_file("./resource/two_dof.urdf")
print("Imported " + p.name.decode("utf-8"))

# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# set q
print("Before", p.mbc.q)
p.mbc.q = [[], [-np.pi/2], [np.pi/2]]
print("After", p.mbc.q)

# set dq
print("Before", p.mbc.alpha)
p.mbc.alpha = [[], [-2.0], [-3.0]]
print("After", p.mbc.alpha)

# forward kinematics
rbd.forwardKinematics(p.mb, p.mbc)
rbd.forwardVelocity(p.mb, p.mbc)

# pose
ofsRot = e.Matrix3d.Identity()
ofsPos = e.Vector3d(0,0,1)
j_X_i = s.PTransformd(ofsRot.transpose(), ofsPos)
j_X_0 = j_X_i * p.mbc.bodyPosW[body_id_from_name("link_2", p.mb.bodies())] # last body
T = sva_to_affine(j_X_0)
print("T:\n", T)

f = T.rows()
print(f)

# jacobian, dJacobian [angular; linear]
bodyName = "link_2"
jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
J = jac.jacobian(p.mb, p.mbc)
dJ = jac.jacobianDot(p.mb, p.mbc)
print("Jacobian:\n", J)
print("dJacobian:\n", dJ)

# mass matrix
fd = rbd.ForwardDynamics(p.mb)
fd.computeH(p.mb, p.mbc);
M = fd.H()
print("MassMatrix (M):\n", M)

# nonlinear effects
fd = rbd.ForwardDynamics(p.mb)
fd.computeC(p.mb, p.mbc);
h = fd.C()
print("NonlinearEffects (h):\n", h)

# coriolis matrix
cor = rbd.Coriolis(p.mb)
C = cor.coriolis(p.mb, p.mbc)
print("Coriolis matrix (C, where c = C*dq):\n", C)
 
print_multi_body(p.mb)

Imported TwoJointRobot
Before [[], [0.0], [0.0]]
After [[], [-1.5707963267948966], [1.5707963267948966]]
Before [[], [0.0], [0.0]]
After [[], [-2.0], [-3.0]]
T:
           1           0           0 6.12323e-17
          0 3.26795e-07          -1          -1
          0           1 3.26795e-07          -1
          0           0           0           1
4
Jacobian:
            0            0
          -1           -1
 3.26795e-07  3.26795e-07
           1 -1.11022e-16
 2.00104e-23            0
 6.12323e-17            0
dJacobian:
 -2.11758e-22 -2.11758e-22
           0            0
           0            0
 1.22465e-16            0
 -6.5359e-07  3.62815e-23
          -2  1.11022e-16
MassMatrix (M):
 4.00002 1.00001
1.00001 1.00001
NonlinearEffects (h):
 -11.19
 13.81
Coriolis matrix (C, where c = C*dq):
  3  5
-2  0
number of bodies: 3
number of joints: 3
bodies:
body index: 0 body name: base_link
body index: 1 body name: link_1
body index: 2 body name: link_2
joints:
joint index: 0 joi

#### Operational Space Control of 4 dof system with 2 tasks
The following example controls an RRRR manipulator.
<br> Task 1 - control of the xz coordinates of the end effector
<br> Task 2 - control of the xz coordinates of the elbow

Note that the second task is made singular (by specifying a desired position that is very far away). The inverse SPD function will mask the singular direction when computing the corresponding $\Lambda$. The secondary task is redundant if we consider the direction orthogonal to the singular direction as our controllable direction. The 'singular' direction can be projected into the third task to provide joint space damping.

In [4]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms

def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation()
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row])
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))

def inv_spd(mat, tolerance=1e-4):
    '''Converts an inverse into an SVD problem with singularity handling
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    u, s, vt = np.linalg.svd(mat_np, full_matrices=True, hermitian=True)

    sinv = np.zeros(s.shape)
    half_tolerance = tolerance/2;
     
    for i in range(len(s)):
        if s[i] > tolerance:
            sinv[i] = 1/s[i]
        elif s[i] > half_tolerance:
            sinv[i] = (s[i]-half_tolerance)/half_tolerance * (1/tolerance)
        else:
            sinv[i] = 0

    return e.MatrixXd(vt.transpose()@np.diag(sinv)@u.transpose())

# import robot as global variable
p = from_urdf_file("./resource/four_dof.urdf")
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.SHARED_MEMORY)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF("./resource/four_dof.urdf",startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# setup configuration
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = [0.6524786685190415, -0.7891557752882681, 0.8792139557391727, 1.3774730897711231]
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
    
# simulate torque control
for i in range(100000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # read state feedback
    q = [joint_states[i][0] for i in jointsId]
    dq = [joint_states[i][1] for i in jointsId]
    
    # update state in model
    p.mbc.q = [[], [q[0]], [q[1]], [q[2]], [q[3]]]
    p.mbc.alpha = [[], [dq[0]], [dq[1]], [dq[2]], [dq[3]]]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(1,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name("link_4", p.mb.bodies())])
    T2 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name("link_2", p.mb.bodies())])
    
    # select xz coordinates
    S = e.MatrixXd.Zero(2,6)
    S.coeff(0,3,1)
    S.coeff(1,5,1)
    
    # jacobian, dJacobian [angular; linear]
    bodyName = "link_4"
    jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S*tempJ1
    dJ1 = S*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    bodyName = "link_2"
    jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
    tempJ2 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ2 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ2);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ2);
    J2 = S*tempJ2
    dJ2 = S*tempdJ2
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-3)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1
    
    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
    # change desired position for secondary task
    if i < MOVEMENT_TIME:
        xdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
        zdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
    
    # error
    err1 = e.VectorXd(2)
    err1[0] = 2 - T1.coeff(0,3)
    err1[1] = 2 - T1.coeff(2,3)
    derr1 = -J1*dq
    err2 = e.VectorXd(2)
    err2[0] = xdes - T2.coeff(0,3)
    err2[1] = zdes - T2.coeff(2,3)
    derr2 = -J2*dq

    # control
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(400*err1+40*derr1-dJ1*dq))
    tau_2 = e.VectorXd(J21.transpose()*lambda21*(400*err2+40*derr2-dJ2*dq-J2*Minv*tau_1))
    tau = tau_1 + Nprec1.transpose()*tau_2 + h

    # command system:
    for i in jointsId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

Imported FourJointRobot


error: Not connected to physics server.